# Advent of Code 2022: Day 18
https://adventofcode.com/2022/day/18


## Part 1
Calculate the number of exposed sides on a 3d object made of 1x1x1 cubes with coordinates x,y,z.

### Get the data into a list of strings

In [ ]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Function to preprocess the data

In [ ]:
def preprocess_data(data):
  
  # Setup the list and dictionary
  # to output. 
  output_list = []
  side_dict = {}
  
  # Go through each cube in the data
  # and get the three coordinate numbers
  # into a list. Make an entry in the 
  # dictionary, with the original string
  # as key, and the value being another
  # dictionary, containing the list of
  # numbers, along with an indicator
  # for each of the 6 sides, initialized
  # as being true.
  for item in data:
    temp_list = []
    split = item.split(',')
    for s in split:
      temp_list.append(int(s))
    side_dict[item] = {'sides': temp_list,
                       'left': True,
                       'right': True,
                       'up': True,
                       'down': True,
                       'forward': True,
                       'backward':True}
    
    output_list.append(temp_list)
  output_list.sort()
  return side_dict, output_list

### Function to compare two cubes

In [ ]:
def compare_cubes(side_dict, cube1, cube2):
  
  # Get the list of numbers
  # for each cube.
  c1 = side_dict[cube1]['sides']
  c2 = side_dict[cube2]['sides']

  # Check for X if both not already false
  if side_dict[cube1]['right'] == True or side_dict[cube1]['left'] == True: 
    if (c1[1] == c2[1]) and (c1[2] == c2[2]):
      if c1[0]-c2[0] == -1:
        side_dict[cube1]['right'] = False
        side_dict[cube2]['left'] = False
      elif c1[0]-c2[0] == 1:
        side_dict[cube1]['left'] = False
        side_dict[cube2]['right'] = False

  # Check for Y if both not already false
  if side_dict[cube1]['up'] == True or side_dict[cube1]['down'] == True: 
    if (c1[0] == c2[0]) and (c1[2] == c2[2]):
      if c1[1]-c2[1] == -1:
        side_dict[cube1]['down'] = False
        side_dict[cube2]['up'] = False
      elif c1[1]-c2[1] == 1:
        side_dict[cube1]['up'] = False
        side_dict[cube2]['down'] = False
  
  # Check for Z if both not already false
  if side_dict[cube1]['forward'] == True or side_dict[cube1]['backward'] == True: 
    if (c1[0] == c2[0]) and (c1[1] == c2[1]):
      if c1[2]-c2[2] == -1:
        side_dict[cube1]['forward'] = False
        side_dict[cube2]['backward'] = False
      elif c1[2]-c2[2] == 1:
        side_dict[cube1]['backward'] = False
        side_dict[cube2]['forward'] = False
  
  return side_dict

### Function to get a list of neighboring positions for each cube

In [ ]:
def get_neighbors(cube_sides):
  neighbors = []
  for i in range(3):
    if i == 0:
      c = cube_sides.copy()
      c[0] = c[0]-1
      neighbors.append(c)
      c = cube_sides.copy()
      c[0] = c[0]+1
      neighbors.append(c)
    if i ==1:
      c = cube_sides.copy()
      c[1] = c[1]-1
      neighbors.append(c)
      c = cube_sides.copy()
      c[1] = c[1]+1
      neighbors.append(c)
    if i ==2:
      c = cube_sides.copy()
      c[2] = c[2]-1
      neighbors.append(c)
      c = cube_sides.copy()
      c[2] = c[2]+1
      neighbors.append(c)
  return neighbors

### Simple DFS algorithm to see if the side of a cube is truly exposed to the outside

In [ ]:
def walk_out(cube_list, start, goals = [0,20]):
  
  # Setup explore and visited
  # lists and an indicator for
  # if the goal was found. 
  exploreQ = []
  exploreQ.append(start)
  visited = []
  visited.append(start)
  found_goal = False

  while exploreQ and not found_goal:
    
    # Get a position to explore from and 
    # retrieve all its neighboring positions. 
    curr = exploreQ.pop(0)
    neighbors = get_neighbors(curr)
    for neighbor in neighbors:
      
      # If a neighboring position has not already 
      # been visited and is not a cube, add it to
      # the explore and visited lists. To ensure
      # the function will never go infinite, also
      # check that a neighbor cannot be outside
      # the goal limits.
      if (neighbor not in visited) and (str(neighbor).rstrip(']').lstrip('[').replace(' ','') not in cube_list) and (min(neighbor) >= goals[0]) and (max(neighbor)<=goals[1]):
        exploreQ.append(neighbor)
        visited.append(neighbor)

      # If any of the x,y,z coordinates reaches the goal
      # limits, then the side is truly exposed to the outside.
      if neighbor[0] in goals or neighbor[1] in goals or neighbor[2] in goals:
        found_goal = True
        break
             
  return found_goal

### Function to count the number of exposed sides.

In [ ]:
def count_sides(data, check_voids = False):
  
  # Setup the dictionary and list of numbers.
  # Also get a list of the dictionary keys. 
  s,l = preprocess_data(data)
  keys = list(s.keys())  
  
  # Compare every cube and mark touching
  # sides as False. 
  for i in range(len(s)):
    for j in range(i+1, (len(s))):
      cube1 = keys[i]
      cube2 = keys[j]
      s = compare_cubes(s, cube1, cube2)
  
  # If the function should also check for
  # empty spaces inside the cube (voids)
  if check_voids == True:
    
    # Get a list of the 6 sides of a cube.
    inner_keys = list(s[keys[0]].keys())[1:]
    
    # Go through every side
    # of every cube. 
    for i in range(len(s)):
      for j in range(len(inner_keys)):
        curr_inner_key = inner_keys[j]
        
        # If the side is still marked as True
        # perform the DFS from the position
        # to the corresponding side of the cube.
        # Mark the side as False if the DFS
        # did not find a way outside the cube.
        if s[keys[i]][curr_inner_key] == True:
          c = s[keys[i]]['sides'].copy()
          if curr_inner_key == 'left':
              c[0] -=1
              free = walk_out(keys, c)
              if not free:
                s[keys[i]][curr_inner_key] = False

          if curr_inner_key == 'right':
              c[0] +=1
              free = walk_out(keys, c)
              if not free:
                s[keys[i]][curr_inner_key] = False

          if curr_inner_key == 'up':
              c[1] -=1
              free = walk_out(keys, c)
              if not free:
                s[keys[i]][curr_inner_key] = False

          if curr_inner_key == 'down':
              c[1] +=1
              free = walk_out(keys, c)
              if not free:
                s[keys[i]][curr_inner_key] = False

          if curr_inner_key == 'forward':
              c[2] +=1
              free = walk_out(keys, c)
              if not free:
                s[keys[i]][curr_inner_key] = False 

          if curr_inner_key == 'backward':
              c[2] -=1
              free = walk_out(keys, c)
              if not free:
                s[keys[i]][curr_inner_key] = False              

  # Count the number of 
  # exposed sides.
  side_count = 0
  inner_keys = list(s[keys[0]].keys())[1:]
  for i in range(len(s)):
    for j in range(len(inner_keys)):
      if s[keys[i]][inner_keys[j]] == True:
        side_count += 1
  return side_count

In [ ]:
count_sides(data_list)

3636

## Part 2
Calculate the number of exposed sides on a 3d object made of 1x1x1 cubes with coordinates x,y,z, only if they are truly exposed to the outside. This means that sides exposed to an empty space that is completely surrounded by cubes should not count as being exposed.

In [ ]:
count_sides(data_list, True)

2102